# Decision Tree with pure Python (Without External Library)

Python and OOP

##### Train Data

In [3]:
training_data = [['Green',3,'Apple'],
                 ['Yellow',3,'Apple'],
                 ['Red',1,'Grape'],
                 ['Red',1,'Grape'],
                 ['Yellow',3,'Lemon']] #TOY DATASET

##### Column Labels
These are used to print tree

In [4]:
header = ["Color","Diameter","label"]

#####  Finds Unique values for a column in dataset

In [5]:
def unique_vals(Data,col): #training_data,0
    return set([row[col] for row in Data])

In [6]:
unique_vals(training_data,2)

{'Apple', 'Grape', 'Lemon'}

demo

In [7]:
{'Apple':2,'Grape':2,'Lemon':1}

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

##### Count the number of each type in dataset

In [8]:
def class_counts(Data):
    counts = {}
    for row in Data:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

demo

In [9]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

##### A Question is used to partition a dataset.

This class just records a 'colum number (e.g., 0 for Color) and a column value (e.g., Green). The match method is used to compare the feature value in an example to the feature value stored in the question. See the demo below.

In [10]:
class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
    def match(self,example): #example means row --> ['Green',3,'Apple']
        val = example[self.column]
        return val == self.value #'Green' == 'Red', returns a boolean
    def __repr__(self):
        return "Is %s %s %s?" % (header[self.column],"==",str(self.value))

In [11]:
Question(0,"Red")

Is Color == Red?

In [12]:
q = Question(1,4)
print(q)

Is Diameter == 4?


In [13]:
q.match(training_data[0])

False

##### Partitions a dataset.

For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows'.

In [14]:
def partition(Data,question):
    true_rows,false_rows = [],[]
    for row in Data: #row is also called example --> ['Green',3,'Apple']
        if(question.match(row)):
            true_rows.append(row) # --> [['Green',3,'Apple']]
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [15]:
true_rows,false_rows = partition(training_data,
                                 Question(0,'Green'))
print('True Rows: ',true_rows)
print('False Rows: ',false_rows)

True Rows:  [['Green', 3, 'Apple']]
False Rows:  [['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]


##### Calculate the Gini Impurity for a list of rows.

There are a few different ways to do this, I thought this one was the most concise. See: https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity

In [16]:
def gini(Data):
    counts = class_counts(Data)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(Data))
#         print(prob_of_lbl)
        impurity-=prob_of_lbl**2
#         print(impurity)
    return impurity

In [17]:
gini(training_data)

0.6399999999999999

##### Information Gain.

The uncertainty of the starting node, minus the weighted impurity of two child nodes.

In [18]:
def info_gain(left,right,current_impurity): #current impurity means GDS 
    #left means true, right means false
    p = float(len(left))/(len(left)+len(right)) #prob of true rows
    return current_impurity - p*gini(left) - (1-p)*gini(right)

In [19]:
true_rows,false_rows = partition(training_data,
                            Question(1,3))

In [20]:
info_gain(true_rows,false_rows,gini(training_data))

0.37333333333333324

###### Find the best question to ask by iterating over every feature / value and calculating the information gain.

In [27]:
def find_best_split(Data):
    best_gain = 0
    best_question = None
    current_impurity = gini(Data) #Gds
    n_features = len(Data[0]) - 1
    for col in range(n_features): #0
        values = unique_vals(Data,col) #[Green,Red,Yellow]
        for val in values:
            question = Question(col,val)
            true_rows,false_rows = partition(Data,question)
            if(len(true_rows) == 0 or len(false_rows)==0):
                continue
            gain = info_gain(true_rows,
                             false_rows,
                             current_impurity)
            if gain>=best_gain:
                best_gain, best_question = gain , question
    return best_gain,best_question

In [28]:
best_gain,best_question = find_best_split(training_data)
print(best_question)
print(best_gain)

Is Diameter == 3?
0.37333333333333324


##### A Leaf node classifies data.

This holds a dictionary of class (e.g., "Mango") -> number of times it appears in the rows from the training data that reach this leaf.

In [70]:
class Leaf:
    def __init__(self,Data):
        self.predictions = class_counts(Data)

##### A Decision Node asks a question.

This holds a reference to the question, and to the two child nodes.

In [71]:
class Decision_Node:
    def __init__(self, question, true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        #print(self.question)

##### Builds the tree.

Try partitioing the dataset on each of the unique attribute, 
calculate the information gain, 
and return the question that produces the highest gain.

In [72]:
def build_tree(Data,i=0):
    gain, question = find_best_split(Data) #FIND BEST QUESTION
    

    # Base case: no further info gain 
    # since we can ask no further questions,
    # we'Ll return a leaf.
    if gain == 0:
        return Leaf(Data)
    
    # If we reach here, we have found a useful feature / value 
    # to partition on.
    true_rows , false_rows = partition(Data,question)
    
    true_branch = build_tree(true_rows,i)
    false_branch = build_tree(false_rows,i)
    false_branch = build tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point, 
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question,true_branch,false_branch)

In [73]:
my_tree = build_tree(training_data)
print(my_tree)

In [74]:
def print_tree(node,spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Predict",node.predictions)
        return
    print(spacing+str(node.question))
    print(spacing + "--> True:")
    print_tree(node.true_branch , spacing + "\t")
    
    print(spacing + "--> False:")
    print_tree(node.false_branch , spacing + "\t")
    

In [75]:
print_tree(my_tree)

Is Diameter == 3?
--> True:
	Is Color == Yellow?
	--> True:
		Predict {'Lemon': 1, 'Apple': 1}
	--> False:
		Predict {'Apple': 1}
--> False:
	Predict {'Grape': 2}


In [76]:
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total * 100)) + "%"
    return probs

In [77]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    
    # Decide whether to follow the true-branch or the false-branch.
    # Compate the feature / value stored in the node, 
    # to the example we're considering.
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

##### Test Data

In [28]:
testing_data = [
    ["Red",1,"Apple"],
    ["Yellow" , 3 , "Apple"]
]

##### Comparing actual and predicted value

In [29]:
for row in testing_data:
    print("Actual: %s. Predicted: %s" % 
          (row[-1],print_leaf(classify(row,my_tree))))

Actual: Apple. Predicted: {'Grape': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}


##### Training model with 2nd dataset to increase accuracy

In [90]:
header=["outlook","temperature","humidity","wind","decision"]

training_data2= [
['sunny','hot','high','weak','no'],
['sunny','hot','high','strong','no'],
['overcast','hot','high','weak','yes'],
['rain','mild','high','weak','yes'],
['rain','cool','normal','weak','yes'],
['rain','cool','normal','strong','no'],
['overcast','cool','normal','strong','yes'],
['sunny','mild','high','weak','no'],
['sunny','cool','normal','weak','yes'],
['rain','mild','normal','weak','yes'],
['sunny','mild','normal','strong','yes'],
['overcast','mild','high','strong','yes'],
['overcast','hot','normal','weak','yes'],
['rain','mild','high','strong','no'],
]

##### Build the Decision tree with "training_data2" dataset

In [91]:
my_tree2 = build_tree(training_data2)

#### Print The Final Tree

In [92]:
print_tree(my_tree2)

Is outlook == overcast?
--> True:
	Predict {'yes': 4}
--> False:
	Is humidity == high?
	--> True:
		Is outlook == sunny?
		--> True:
			Predict {'no': 3}
		--> False:
			Is wind == strong?
			--> True:
				Predict {'no': 1}
			--> False:
				Predict {'yes': 1}
	--> False:
		Is wind == strong?
		--> True:
			Is temperature == cool?
			--> True:
				Predict {'no': 1}
			--> False:
				Predict {'yes': 1}
		--> False:
			Predict {'yes': 3}


In [93]:
testing_data2 = ["overcast","mild","normal","weak"]

In [94]:
classify(testing_data2,my_tree2)

{'yes': 4}

In [95]:
print("Predicted: %s" % (print_leaf(classify(testing_data2,my_tree2))))

Predicted: {'yes': '100%'}
